### Linear Gaussian Model

In [2]:
import numpy as np 

#### Auxiliary functions

In [3]:
# Skeleton definition
NUI_SKELETON_POSITION_COUNT = 20

NONE = -1
HIP_CENTER = 0
SPINE = 1
SHOULDER_CENTER = 2
HEAD = 3
SHOULDER_LEFT = 4
ELBOW_LEFT = 5
WRIST_LEFT = 6
HAND_LEFT = 7
SHOULDER_RIGHT = 8
ELBOW_RIGHT = 9
WRIST_RIGHT = 10
HAND_RIGHT = 11
HIP_LEFT = 12
KNEE_LEFT = 13
ANKLE_LEFT = 14
FOOT_LEFT = 15
HIP_RIGHT = 16
KNEE_RIGHT = 17
ANKLE_RIGHT = 18
FOOT_RIGHT = 19

nui_skeleton_names = ( \
    'HIP_CENTER', 'SPINE', 'SHOULDER_CENTER', 'HEAD', \
    'SHOULDER_LEFT', 'ELBOW_LEFT', 'WRIST_LEFT', 'HAND_LEFT', \
    'SHOULDER_RIGHT', 'ELBOW_RIGHT', 'WRIST_RIGHT', 'HAND_RIGHT', \
    'HIP_LEFT', 'KNEE_LEFT', 'ANKLE_LEFT', 'FOOT_LEFT', \
    'HIP_RIGHT', 'KNEE_RIGHT', 'ANKLE_RIGHT', 'FOOT_RIGHT' )

nui_skeleton_conn = ( \
    NONE, \
    HIP_CENTER, \
    SPINE, \
    SHOULDER_CENTER, \
    # Left arm 
    SHOULDER_CENTER, \
    SHOULDER_LEFT,  \
    ELBOW_LEFT,  \
    WRIST_LEFT,  \
    # Right arm 
    SHOULDER_CENTER,  \
    SHOULDER_RIGHT,  \
    ELBOW_RIGHT,  \
    WRIST_RIGHT,  \
    # Left leg 
    HIP_CENTER,  \
    HIP_LEFT,  \
    KNEE_LEFT,  \
    ANKLE_LEFT,  \
    # Right leg 
    HIP_CENTER,  \
    HIP_RIGHT,  \
    KNEE_RIGHT,  \
    ANKLE_RIGHT,  \
)


In [36]:
def load_dataset(file=None):
    """
      Returns the data, the labels and the person id for each action
    """
    import scipy.io
    
    if file is None:
        ex = scipy.io.loadmat('data/data.mat')
    else:
        ex = scipy.io.loadmat(file)
        
    return ex['data'],ex['labels'],ex['individuals']


In [ ]:
"""
    To see if the data is loaded and to be familiar with data 
    also to have general idea of the the dimensions
""" 
data , labels, individuals = load_dataset()
print(data.shape)
print(labels.shape)
# print(individuals.shape)
# print(data)

# to see the classes of the joints
for cls in np.nditer(labels):
    print(cls, end='  ')

In [38]:
def log_normpdf(x, mu, sigma):
    """
      Computes the natural logarithm of the normal probability density function
      
    """
pass


In [39]:
def normalize_logprobs(log_probs):
    """
       Returns the log prob normalizes so that when exponenciated
       it adds up to 1 (Useful to normalizes logprobs)
    """
    mm = np.max(log_probs)
    return log_probs - mm - np.log(np.sum(np.exp(log_probs - mm)))


#### Functions to implement

In [40]:
def fit_gaussian(X, W=None):
    """
      Compute the mean and variance of X, 
      You can ignore W for the moment
    """
    #mean
    mean = X.mean()
    #variance
    variance = X.var()
    return (mean, variance)


In [16]:
print(data[1,:,:])
# for x in np.nditer(data):
#     print(x)
#     print(fit_gaussian(x))
indexs = np.where(labels == 8)
cl_data = data[:,:,indexs[0]]
print(cl_data.shape)
    

[[ 0.08341   0.081944  0.082453 ...  0.007845 -0.06222  -0.068802]
 [ 0.251406  0.258025  0.25243  ...  0.195303  0.183259  0.133769]
 [ 2.70917   2.698782  2.685623 ...  2.568254  2.507551  2.61902 ]]
(20, 3, 515)


In [41]:
def my_cov(x,y,w):
    """
      Useful function for fit_linear_gaussian
    """
    return np.sum(w*x*y)/np.sum(w)-np.sum(w*x)*np.sum(w*y)/np.sum(w)/np.sum(w)


In [42]:
def fit_linear_gaussian(Y,X,W = None):
    """
    Input:
      Y: vector of size D with the observations for the variable
      X: matrix DxV with the observations for the parent variables
                 of X. V is the number of parent variables
                 thus X has observations of Y's parents
      W: vector of size D with the weights of the instances (ignore for the moment)
      
    Outout:
    
       The betas and sigma
    """
    
        # no parents
    len_betas = X.shape[1]+1;
    b = np.zeros(len_betas)
    A = np.zeros((len_betas,len_betas))
    betas = np.zeros(len_betas)
    sigma = 0
    
    # FORMAT b , from b = Ax..
    
    # Constract vector b .. the interest variable.
    for i in range(len_betas):
        if i==0:
            b[0] = np.mean(Y)        
        else:
            b[i] = np.mean(X[:,i-1]*Y)
    
    # Format A matrix.
    A[0,0] = 1
    # first row and first col of A
    for i in range(1,len_betas):
        A[0,i] = A[i,0] = np.mean(X[:,i-1])
        
    # For the rest
    for i in range(1,len_betas):
        for j in range(1,len_betas):
            A[i,j] = np.mean(X[:,i-1]*X[:,j-1])
    # to find betas we can use linear equation: b = Ax=> x = inverse(A)b
    betas = np.linalg.solve(A,b)
    
    # sigmas and 
    w = 1
    sigma_cov = 0 
    for i in range(1,X.shape[1]):
        for j in range(1,X.shape[1]):
            sigma_cov += betas[i]*betas[j]*my_cov(X[:,i],X[:,j],w)

    sigma = my_cov(Y, Y , 1) - sigma_cov;  
    
    # but how do we get b: 
    betas = np.linalg.solve(X,Y)
    
    
    return (betas,sigma)

In [43]:
def prior_prob(labels):
    
    unique_labels =np.unique(labels)
    prior = np.zeros(unique_labels.size)
    for i in range(unique_labels.size):
        prior[i] = (np.size(labels[labels == unique_labels[i]])/labels.size)
    return prior

In [67]:
def learn_model(dataset, labels, G=None):
    """
    Input:
     dataset: The data as it is loaded from load_data
     labels:  The labels as loaded from load_data
     Graph:   (optional) If None, this def should compute the naive 
           bayes model. If it contains a skel description (pe 
           nui_skeleton_conn, as obtained from skel_model) then it should
           compute the model using the Linear Gausian Model

    Output: the model
     a (tentative) structure for the output model is:
       model.connectivity: the input Graph variable should be stored here 
                           for later use.
       model.class_priors: containing a vector with the prior estimations
                           for each class
       model.jointparts[i] contains the estimated parameters for the i-th joint

          For joints that only depend on the class model.jointparts(i) has:
            model.jointparts(i).means: a matrix of 3 x #classes with the
                   estimated means for each of the x,y,z variables of the 
                   i-th joint and for each class.
            model.jointparts(i).sigma: a matrix of 3 x #classes with the
                   estimated stadar deviations for each of the x,y,z 
                   variables of the i-th joint and for each class.

          For joints that follow a gausian linear model model.jointparts(i) has:
            model.jointparts(i).betas: a matrix of 12 x #classes with the
                   estimated betas for each x,y,z variables (12 in total) 
                   of the i-th joint and for each class label.
            model.jointparts(i).sigma: as above
            https://www.youtube.com/watch?v=PErqizZqLjI
    """
    # estimate priori probablity of classes and create model object
    prioriProb = prior_prob(labels)
    model = Model(G, prioriProb)
    
    # for each classes 
    cls = np.array([1,2,3,8],np.int32)
    for cl in np.nditer(cls):
        indexs = np.where(labels == cl)
        cl_data = data[:,:,indexs[0]]
        cl_index = np.where(cls == cl)[0][0] # for indexing purpose
        
        # we have 3 coordinate for each joints(20 Jnts)
        for jointCord in range(20): 
            if G is None or G[joint] == -1:
                (mean_x, variance_x) = fit_gaussian(cl_data[jointCord,0,:]) #x
                (mean_y, variance_y) = fit_gaussian(cl_data[jointCord,1,:]) #y
                (mean_z, variance_z) = fit_gaussian(cl_data[jointCord,2,:]) #z
                mean = np.array([mean_x,mean_y,mean_z])
                variance = np.array([variance_x, variance_y, variance_z])
                model.jointParts(jointCord,cl_index,mean = mean.T,variance = variance.T)
                
            else:
                (betas_x, sigma_x) = fit_linear_gaussian(cl_data[jointCord,0,:],cl_data[G[jointCord],:,:]) #x
                (betas_y, sigma_y) = fit_linear_gaussian(cl_data[jointCord,1,:],cl_data[G[jointCord],:,:]) #y
                (betas_z, sigma_z) = fit_linear_gaussian(cl_data[jointCord,2,:],cl_data[G[jointCord],:,:]) #z
                betas = np.append(betas_x, betas_y, betas_z)
                sigma = np.append(sigma_x, sigma_y,sigma_z)
                model.jointParts(jointCord,cl_index,betas = betas.T,sigma = sigma.T)
                
    return model

In [58]:
class Model:
    def __init__(self,connectivity,priori_prob):
        self.mean = np.zeros((20,3,4))
        self.variance = np.zeros((20,3,4))
        self.betas = np.zeros((20,12,4))
        self.sigma= np.zeros((20,12,4))
        self.connectivity = connectivity
        self.priori_prob = priori_prob
        
    
    # Input graph variables
    def getConnectivity(self):
        return self.connectivity 
    
    # Priori probablity of each class
    def getClass_priori(self, priori_prob):
        self.priori_prob = priori_prob 
    
    # estimated parameters of each joint
    def jointParts(self, index,cl,mean = None, variance  = None, betas  = None, sigma  = None):

        self.index = index
        if self.connectivity is None or self.connectivity(index) == -1:
            self.mean[index,:,cl] = mean # matrix of 20 x 3 x #classes
            self.variance[index,:,cl] = variance # matrix of 20 x 3 x #classes
        else:
            self.betas[index,:,cl] = betas # matrix of 20 x 12 x #classes
            self.sigma[index,:,cl] = sigma # matrix of 20 x 12 x #classes
                

In [ ]:
# Test area

model = Model(np.zeros((12,1)),np.zeros((12,1)))
print(model.mean.shape)

model = learn_model(data,labels)
print(model.mean)

In [6]:
def classify_instances(instances, model):
    """    
    Input
       instance: a 20x3x#instances matrix defining body positions of
                 instances
       model: as the output of learn_model

    Output
       probs: a matrix of #instances x #classes with the probability of each
              instance of belonging to each of the classes

    Important: to avoid underflow numerical issues this computations should
               be performed in log space
    """
    
    
    return

In [ ]:
def compute_logprobs(example, model):
    """
       Input
           instance: a 20x3 matrix defining body positions of one instance
           model: as given by learn_model

       Output
           l: a vector of len #classes containing the loglikelihhod of the 
              instance

    """
    return
